## ☑️ Combining multiple tables

- Complete the following questions
- Make sure you run the following code cells before you attempt any of the questions
- We will work with `flights.db` database through this workbook

First, import pandas and sqlite3 libraries and create the connection to the `flights.db` database, located in the `data` folder:

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('data/flights.db')

A database might have multiple tables. It's a good idea to do an initial exploration of the database by first querying the `sqlite_master` table to see what tables are in the database.

Run the following code cell to show all the tables in the `flights.db` database:

In [2]:
query = """
SELECT name 
FROM sqlite_master 
WHERE type = 'table';
"""
df = pd.read_sql_query(query, conn)
df

#df.name.to_list()

,name
0,airports
1,airlines
2,routes


Run the following code cell to show the schema for each table in `flights.db` database:

In [3]:
for table in ['airports','airlines','routes']:
    
    query = f"""
    SELECT sql 
    FROM sqlite_master 
    WHERE name = '{table}';
    """
    
    df = pd.read_sql_query(query, conn)
    print(''.join(df.values[0, 0]))

CREATE TABLE airports (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [city] TEXT,
  [country] TEXT,
  [code] TEXT,
  [icao] TEXT,
  [latitude] TEXT,
  [longitude] TEXT,
  [altitude] TEXT,
  [offset] TEXT,
  [dst] TEXT,
  [timezone] TEXT
)
CREATE TABLE airlines (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [alias] TEXT,
  [iata] TEXT,
  [icao] TEXT,
  [callsign] TEXT,
  [country] TEXT,
  [active] TEXT
)
CREATE TABLE routes (
[index] INTEGER,
  [airline] TEXT,
  [airline_id] TEXT,
  [source] TEXT,
  [source_id] TEXT,
  [dest] TEXT,
  [dest_id] TEXT,
  [codeshare] TEXT,
  [stops] TEXT,
  [equipment] TEXT
)


Run the following code cells to show the contents for each table in `flights.db` database: `['airports','airlines','routes']`

In [4]:
query = """
SELECT *
FROM airports;
"""
df = pd.read_sql_query(query, conn)
df.head(2)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby


In [5]:
query = """
SELECT *
FROM airlines;
"""
df = pd.read_sql_query(query, conn)
df.head(2)

,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y
1,1,2,135 Airways,\N,None,GNL,GENERAL,United States,N


In [6]:
query = """
SELECT *
FROM routes;
"""
df = pd.read_sql_query(query, conn)
df.head(2)

,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2
1,1,2B,410,ASF,2966,KZN,2990,None,0,CR2


**Q1)** Combine `routes` and `airlines` tables using a `INNER JOIN`.

- Please note `routes.airline_id` column and `airlines.id` column are related in the two tables: `routes.airline_id=airlines.id`

- Show only `routes.airline`, `routes.airline_id`, `airlines.name` columns in the final result

See below code syntax for some guidance:
```SQL
SELECT <column_names>
FROM <table1>
INNER JOIN <table2>
ON table1.column_name = table2.column_name;
```
Please note you have been provided with the code for this question to carry out the necessary analysis work. Simply run the code cell to produce the desired results.

In [7]:
query = """
SELECT routes.airline, routes.airline_id, airlines.name
FROM routes 
INNER JOIN airlines 
ON routes.airline_id = airlines.id;
""" 
df = pd.read_sql_query(query, conn)
df

,airline,airline_id,name
0,2B,410,Aerocondor
1,2B,410,Aerocondor
2,2B,410,Aerocondor
3,2B,410,Aerocondor
4,2B,410,Aerocondor
...,...,...,...
66980,ZL,4178,Regional Express
66981,ZM,19016,Apache Air
66982,ZM,19016,Apache Air
66983,ZM,19016,Apache Air


**Q2)** Now combine `routes` and `airlines` tables using a `LEFT JOIN`

- Please note `routes.airline_id` column and `airlines.id` column are related in the two tables: `routes.airline_id=airlines.id`

- Show only `routes.airline`, `routes.airline_id`, `airlines.name` columns

See below code syntax for some guidance:
```SQL
SELECT <column_names>
FROM <table1>
LEFT JOIN <table2>
ON table1.column_name = table2.column_name;
```
Please note you have been provided with the code for this question to carry out the necessary analysis work. Simply run the code cell to produce the desired results.

In [8]:
query = """
SELECT routes.airline, routes.airline_id, airlines.name
FROM routes
LEFT JOIN airlines
ON routes.airline_id = airlines.id;
"""
df = pd.read_sql_query(query, conn)
df

,airline,airline_id,name
0,2B,410,Aerocondor
1,2B,410,Aerocondor
2,2B,410,Aerocondor
3,2B,410,Aerocondor
4,2B,410,Aerocondor
...,...,...,...
67658,ZL,4178,Regional Express
67659,ZM,19016,Apache Air
67660,ZM,19016,Apache Air
67661,ZM,19016,Apache Air


It is advisable to take a moment to reflect upon the outcomes of questions 1 and 2, ensuring a thorough understanding of the results. Additionally, make sure to jot down any intriguing observations that arise from the analysis.

In [9]:
#add your notes below

#INNER JOIN indeed produces a result with fewer or equal rows compared to the original tables,
#whereas LEFT JOIN can potentially result in a larger number of rows in the output compared to the left table.


**Q3)** Get the `latitude` and `longitude` for the *origin* of each route, along with the name of the `city` where the origin airport is located.

We suggest dealing with this in incremental steps:
- Use a `INNER JOIN` to combine the `routes` and `airports` tables

- Note `routes.source_id` column and `airlines.id` column should be connected in the two tables: `routes.source_id=airports.id`

- Show only `airports.city`, `airports.longitude`, `airports.latitude` columns

- Limit the results to 5 records only

See below code syntax for some guidance:
```SQL
SELECT <column_names>
FROM <table1>
INNER JOIN <table2>
ON table1.column_name = table2.column_name
LIMIT 5;
```

In [10]:
#add your code below
#query = ...

query = """
SELECT airports.city, airports.longitude, airports.latitude
FROM routes
INNER JOIN airports
ON routes.source_id = airports.id
LIMIT 5;
"""
df = pd.read_sql_query(query, conn)
df

,city,longitude,latitude
0,Sochi,39.956589,43.449928
1,Astrakhan,48.006278,46.283333
2,Astrakhan,48.006278,46.283333
3,Chelyabinsk,61.503333,55.305836
4,Chelyabinsk,61.503333,55.305836


**Q4)** We can also get the destination coordinates (latitude and longitude) by matching *destination* id to the airport id. Add an additional `INNER JOIN` to the previous query, and select the corresponding columns to return the destination coordinates.

Please note you have been provided with the code for this question to carry out the necessary analysis work. Simply run the code cell to produce the desired results.

In [11]:
query = """
SELECT sa.city AS source_city,
    sa.longitude AS origin_lon,
    sa.latitude AS origin_lat,
    da.longitude AS dest_lon,
    da.latitude AS dest_lat
FROM routes

INNER JOIN airports AS sa
ON sa.id = routes.source_id

INNER JOIN airports AS da
ON da.id = routes.dest_id

LIMIT 5;
"""
df = pd.read_sql_query(query, conn)
df

,source_city,origin_lon,origin_lat,dest_lon,dest_lat
0,Sochi,39.956589,43.449928,49.278728,55.606186
1,Astrakhan,48.006278,46.283333,49.278728,55.606186
2,Astrakhan,48.006278,46.283333,43.081889,44.225072
3,Chelyabinsk,61.503333,55.305836,49.278728,55.606186
4,Chelyabinsk,61.503333,55.305836,82.650656,55.012622


**Q5)** Find the number of routes per airline. Optionally see if you can sort the results in descending order. 

We suggest dealing with this in incremental steps:
- Use a `INNER JOIN` to combine the `routes` and `airlines` tables

- Note `routes.airline_id` column and `airlines.id` column should be connected in the two tables: `routes.airline_id=airlines.id`

- Remember to `GROUP BY` using `routes.airline_id` column

- Find the number of routes per airline, consider using `COUNT(*)` function

- Show only `airline_id`, `airlines.name` columns along with the `COUNT` of routes per each airline

- Use `ORDER BY` to sort the results in descending order

- Limit the results to 5 records only



See below code syntax for some guidance:
```SQL
SELECT <column_names>, COUNT(*) 
FROM <table1>
INNER JOIN <table2>
ON table1.column_name = table2.column_name
GROUP BY <column_name>
ORDER BY <column_name> DESC
LIMIT 5;
```

In [12]:
#add your code below

query = """
SELECT routes.airline_id, airlines.name, COUNT(*) AS nroutes
FROM routes
INNER JOIN airlines
ON routes.airline_id=airlines.id
GROUP BY routes.airline_id
ORDER BY nroutes DESC
LIMIT 5;
"""
df = pd.read_sql_query(query, conn)
df

,airline_id,name,nroutes
0,4296,Ryanair,2484
1,24,American Airlines,2354
2,5209,United Airlines,2180
3,2009,Delta Air Lines,1981
4,5265,US Airways,1960


## ☑️ SQL Window Functions

- Complete the following questions
- Make sure you run the following code cells before you attempt any of the questions
- We will work with `flights.db` database through this workbook

First, import pandas and sqlite3 libraries and create the connection to the `flights.db` database, located in the `data` folder:

In [13]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('data/flights.db')

Run the following code cell to show all the tables in the `flights.db` database:

The flights database contains three tables:  `airports, airlines, and routes`:

In [14]:
query = """
SELECT name 
FROM sqlite_master 
WHERE type = 'table';
"""
df = pd.read_sql_query(query, conn)
df


,name
0,airports
1,airlines
2,routes


**Q6)** Refer to the `airports` table. For each `timezone`, **rank** each airport by their `latitude` (ascending).

We suggest dealing with this in incremental steps:

- Show only `name`, `city`, `country` columns from the `airports` table

- Remember to use `RANK()` function alongside `OVER()` keyword with the arguments: `PARTITION BY` and `ORDER BY`

- Note the rows should be partition by `timezone` and sorted in ascending order of `latitude`

See below code syntax for some guidance:
```SQL
SELECT <column_names>
RANK() OVER(PARTITION BY <column_name> ORDER BY <column_name> ASC)
FROM airports;
```

In [17]:
#add your code below
#query = ...

query = """
SELECT name, city, country, 
RANK() OVER (PARTITION BY timezone ORDER BY latitude ASC) AS Ranked
FROM airports;
"""
df = pd.read_sql_query(query, conn)
df

,name,city,country,Ranked
0,San Pedro,San Pedro,Cote d'Ivoire,1
1,Abidjan Felix Houphouet Boigny Intl,Abidjan,Cote d'Ivoire,2
2,Daloa,Daloa,Cote d'Ivoire,3
3,Yamoussoukro,Yamoussoukro,Cote d'Ivoire,4
4,Man,Man,Cote d'Ivoire,5
...,...,...,...,...
8102,Layang Layang Airport,Layang Layang Atoll,Malaysia,88
8103,Uummannaq Heliport,Uummannaq,Greenland,89
8104,Dillant Hopkins Airport,Keene,United States,90
8105,Vilamendhoo,Vilamendhoo,Maldives,91


**Q7)** For each airline, get the number of flights that airline runs that leave per country.

For example, if "Adria Airways" have flights from two different cities in "Albania", there should be one row "Adria Airways, Albania, 2".

Please note you have been provided with the code for this question to carry out the necessary analysis work. Simply run the code cell to produce the desired results.

In [16]:
query = """
SELECT DISTINCT al.name, ap.country,
COUNT(*) OVER(PARTITION BY al.name, ap.country ORDER BY al.name ) AS num_flights
FROM routes
INNER JOIN airports AS ap ON ap.id = routes.source_id
INNER JOIN airlines AS al ON al.id = routes.airline_id
LIMIT 50
"""
df = pd.read_sql_query(query, conn)
df.head(10)

,name,country,num_flights
0,40-Mile Air,United States,2
1,ABSA - Aerolinhas Brasileiras,Denmark,1
2,ABSA - Aerolinhas Brasileiras,Norway,5
3,ALAK,Sudan,8
4,Abaet,Brazil,3
5,Abu Dhabi Amiri Flight,Canada,52
6,Adria Airways,Albania,2
7,Adria Airways,Austria,1
8,Adria Airways,Belgium,1
9,Adria Airways,Bosnia and Herzegovina,1
